In [1]:
import pandas as pd

In [35]:
def yield_results():
    with open('input.txt') as f:
        pass_dict = {}
        for line in f:
            if line == '\n':
                yield pd.Series(pass_dict)
                pass_dict = {}
            
            else:
                pairs = line.split()
                for pair in pairs:
                    key, val = pair.split(':')
                    pass_dict[key] = val
        
        yield pd.Series(pass_dict)

In [36]:
df = pd.DataFrame(yield_results())

In [38]:
df

,byr,iyr,eyr,ecl,pid,hcl,hgt,cid
0,2024,2016,2034,zzz,985592671,033b48,181,166
1,2020,2029,1947,zzz,152cm,cfb18a,66cm,NaN
2,NaN,2019,2023,gry,656793259,#888785,177cm,63
3,2018,1936,1928,dne,#5e832a,#7d3b0c,61cm,241
4,1957,2017,2025,oth,597580472,#888785,187cm,247
...,...,...,...,...,...,...,...,...
294,1992,2016,2028,hzl,092856842,#b6652a,189cm,222
295,2023,2025,2005,#be1503,9311657615,a3c52a,182cm,NaN
296,1988,2028,2035,utc,9743739773,#18171d,193cm,128
297,2026,2028,1943,zzz,160cm,z,64cm,74


In [39]:
df[df.cid == '74']

,byr,iyr,eyr,ecl,pid,hcl,hgt,cid
297,2026,2028,1943,zzz,160cm,z,64cm,74


In [48]:
df_valid = df.drop('cid',1).dropna()

In [56]:
df_valid = df_valid[df_valid.byr.astype(str).apply(len) == 4]
df_valid = df_valid[df_valid.iyr.astype(str).apply(len) == 4]
df_valid = df_valid[df_valid.eyr.astype(str).apply(len) == 4]
df_valid = df_valid[df_valid.pid.astype(str).apply(len) == 9]

In [58]:
# byr (Birth Year) - four digits; at least 1920 and at most 2002.
# iyr (Issue Year) - four digits; at least 2010 and at most 2020.
# eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
# hgt (Height) - a number followed by either cm or in:
# If cm, the number must be at least 150 and at most 193.
# If in, the number must be at least 59 and at most 76.
# hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
# ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
# pid (Passport ID) - a nine-digit number, including leading zeroes.
# cid (Country ID) - ignored, missing or not.

In [61]:
df_valid = df_valid[
    (df_valid.byr.astype(int) <= 2002) & (df_valid.byr.astype(int) >= 1920) &
    (df_valid.iyr.astype(int) <= 2020) & (df_valid.iyr.astype(int) >= 2010) &
    (df_valid.eyr.astype(int) <= 2030) & (df_valid.eyr.astype(int) >= 2020)]

In [64]:
df_valid.hgt

4      187cm
6      156cm
7      193cm
12     190cm
13     185cm
       ...  
285    183cm
289    173cm
291    150cm
294    189cm
298    160cm
Name: hgt, Length: 153, dtype: object

In [68]:
# hgt (Height) - a number followed by either cm or in:
# If cm, the number must be at least 150 and at most 193.
# If in, the number must be at least 59 and at most 76.

In [69]:
df_valid = df_valid[df_valid.hgt.str.contains('\d+(cm)|(in)')]

/Users/pstjohn/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [72]:
height = df_valid.hgt.str.extract('(\d+)(\S+)')

In [76]:
cm_valid = (height[height[1] == 'cm'][0].astype(int) >= 150) & (height[height[1] == 'cm'][0].astype(int) <= 193)
in_valid = (height[height[1] == 'in'][0].astype(int) >= 59) & (height[height[1] == 'in'][0].astype(int) <= 76)

In [83]:
df_valid_height = df_valid[df_valid.index.isin(cm_valid[cm_valid].append(in_valid[in_valid]).index)]

In [87]:
ecl = df_valid_height[df_valid_height.hcl.str.contains('#[0-9a-f]{6}') & 
                      df_valid_height.ecl.isin({'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'})]

In [93]:
ecl.pid.astype(int).isna().any()

False

In [94]:
ecl

,byr,iyr,eyr,ecl,pid,hcl,hgt
4,1957,2017,2025,oth,597580472,#888785,187cm
6,1972,2018,2024,amb,349191561,#341e13,156cm
7,1998,2013,2021,brn,408617933,#fffffd,193cm
12,1927,2010,2023,brn,941290886,#c0946f,190cm
13,1926,2011,2026,grn,423210256,#efcc98,185cm
...,...,...,...,...,...,...,...
285,1999,2013,2029,hzl,655501194,#a97842,183cm
289,1948,2016,2021,hzl,176430746,#7d3b0c,173cm
291,1986,2019,2021,hzl,660176034,#c0946f,150cm
294,1992,2016,2028,hzl,92856842,#b6652a,189cm


In [ ]:
# hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
# ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.

# pid (Passport ID) - a nine-digit number, including leading zeroes.